In [1]:
import pandas as pd
import numpy as np
import nltk
import json
from simpletransformers.ner import NERModel
from gensim.models import KeyedVectors
from strsimpy.jaro_winkler import JaroWinkler
from spellchecker import SpellChecker  # pyspellchecker
from nltk.corpus import stopwords

##### Loads NER and W2V models 
source for the NER model : https://ufile.io/ichyycfe (or train it with bert_tagger.ipynb)  
source for the W2V model : https://github.com/eyaler/word2vec-slim/blob/master/GoogleNews-vectors-negative300-SLIM.bin.gz  
source for df_transitions : markov.ipynb

In [2]:
%run constants.ipynb
%run dictionnaries.ipynb

In [3]:
all_tags = get_all_tags()
tagger = NERModel(model_type='bert',
                  model_name=data_directory+'bert/current_model/',
                  labels=all_tags,
                  use_cuda=False)

model_w2v = KeyedVectors.load_word2vec_format(pathword2vec, binary=True)
voc_stopwords = set(stopwords.words('english'))
db = get_DB()

In [4]:
df_transitions = pd.read_csv(bdd_directory+'df_transitions.csv',
                             sep='§',
                             engine='python',
                             index_col=0,
                             encoding='utf-8')
df_transitions.shape

(10, 10)

In [5]:
df_facts = pd.read_csv(bdd_directory+'df_facts.csv',
                       sep='§',
                       engine='python',
                       index_col=0,
                       encoding='utf-8')
df_facts.shape

(104, 2)

##### Functions

In [6]:
%run functions.ipynb
%run tag_to_filter.v1.ipynb

##### Intents

In [7]:
class Intent:
    """ An intent is a function fo a chatbot, there can be multiple one.
    This is a mother class"""

    def __init__(self, name: str, state: str = None):
        if state is None:
            state = json.dumps(init_event(tab=CT_tabs_default))
        self.name = name
        self.interface_in = state
        self.interface_out = None

    def synchronize(self, event: str):
        """ Even when there is no intercation with the chatboat, the dashboard
        can synchronize it to get information back"""

        self.interface_in = event
        self.interface_out = None

    def get_interface_in(self) -> str:
        """ get the traduction of the intput from the interface
        with the dashboard"""

        return self.interface_in

    def get_interface_out(self) -> str:
        """ get the traduction of the output for the interface
        with the dashboard"""

        return self.interface_out

    def interact(self, sentence: str = None) -> str:
        """ This method is called by the chatbot and must return the sentence
        to show in the chat"""

        return "Please implement this method"

##### Markov recommandation

In [8]:
class Predictor:
    """ This class is used to create the recommandation module"""

    def __init__(self, name: str, transitions: pd.DataFrame):
        self.name = name
        self.transitions = transitions

    def predict(self, event: str) -> str:
        """ give the recommanded state"""
        predict = predict_next_state(event, self.transitions)
        return predict

    def random_state(self):
        """ give a random state"""
        return np.random.choice(self.transitions.index)


markov = Predictor(name='Markov', transitions=df_transitions.dropna())
markov.predict(markov.random_state())

'{"filters": {"aircraft": [], "category": [], "company": [], "country": [], "date": ["01092017", "30092017"], "manufacturer": ["airbus", "boeing"]}, "tab": "general"}'

##### creation of the intent displayer-recommandation

In [9]:
class Displayer(Intent):
    """ A displayer is an intent that understand a user sentence and return
    to the dashboard by the interface the board to show and a next
    recommandation """

    def __init__(self,
                 name: str,
                 model_tagger: NERModel,
                 model_predicteur: Predictor,
                 model_w2v: KeyedVectors,
                 db: dict = {},
                 voc_stopwords: set = set(),
                 state: str = None):

        super(Displayer, self).__init__(name)
        self.tagger = model_tagger
        self.predicteur = model_predicteur
        self.w2v = model_w2v
        self.voc_stopwords = voc_stopwords
        self.db = db

    def synchronize(self, event: str):
        self.interface_in = json_string_to_hash(event)
        self.interface_out = self.predicteur.predict(self.interface_in)

    def get_tags(self, sentence: str) -> list:
        """ Associate a tag to each word"""

        sentence_corrected = self.auto_correction(sentence)
        request = self.tagger.predict([sentence_corrected])[0][0]
        print(request, '\n')
        return request

    def get_filters(self, sentence: str) -> dict:
        """ create the state to communicate with the dashboard"""

        request = self.get_tags(sentence)
        tags_values = extract_tags(request)
        filters = tag_to_filters(tags_values)
        filters = apply_date(filters)
        event = {
            CT_tabs: CT_tabs_default,
            CT_filt: filters,
        }
        return event

    def get_output_sentence(self, pred_state_hash: str) -> str:
        """ create a sentence from a state """

        pred_sentence = make_sentence_fom_json(json.loads(pred_state_hash))
        return pred_sentence

    def interact(self, sentence: str) -> str:
        event = self.get_filters(sentence)
        event_state = json.dumps(event)
        self.synchronize(event_state)

        pred_state_hash = self.get_interface_out()
        pred_sentence = self.get_output_sentence(pred_state_hash)

        res = pred_sentence
        return res

    def auto_correction(self, sentence: str):
        """ correction for typos before processing """

        return auto_correction(self.w2v.vocab, db, voc_stopwords, sentence)

In [10]:
Intent_displayer = Displayer(name='Displayer_Recommandation',
                             model_tagger=tagger,
                             model_predicteur=markov,
                             model_w2v=model_w2v,
                             voc_stopwords=voc_stopwords,
                             db=db)
print(Intent_displayer.name)
print(Intent_displayer.get_interface_out())
Intent_displayer.synchronize(Intent_displayer.interface_in)
print(Intent_displayer.get_interface_out())

Displayer_Recommandation
None
{"filters": {"aircraft": ["a350-900"], "category": [], "company": ["air france"], "country": ["england", "france"], "date": ["01092017", "30092017"], "manufacturer": ["airbus"]}, "tab": "general"}


##### Random intent

In [11]:
class Random_info(Intent):
    """ An other intent implemetation which randomly choose a sentences
    from a database if called"""

    def __init__(self, name, db={}):
        super(Random_info, self).__init__(name)
        self.db = db

    def interact(self, sentence: str = None) -> str:
        output = None
        if db:
            k = np.random.choice(list(random_bdd.keys()))
            output = np.random.choice(list(random_bdd[k]["sentences"]))
        return output

In [12]:
random_bdd = {}
for s in list(df_facts['subject'].unique()):
    random_bdd[s] = {}
    random_bdd[s]['sentences'] = set(
        df_facts.loc[df_facts['subject'] == s, 'facts'].values)

Intent_random_info = Random_info(name='Random_Info', db=random_bdd)
print(Intent_random_info.interact())

A woman flying in to Florida tried to smuggle a human head (along with hair, teeth, and skin) from Haiti in order to keep away evil spirits. She was charged with a smuggling a human head into the U.S. without documentation. She was also charged with failure to declare the head and for transporting hazardous material.


##### Intent classifier

In [13]:
class Classifier:
    """ This classifier let the chatbot choose the best intent when
    receiving a user request"""

    def __init__(self, name: str):
        self.name = name

    def predict(self, sentence: str) -> int:
        if sentence.lower() == 'fact':
            c = 1
        else:
            c = 0
        return c


classifier = Classifier(name='Intent classifieur')
print(classifier.predict(sentence=""))

0


In [14]:
intents = {"classifier": classifier,
           "intents": {0: Intent_displayer,
                       1: Intent_random_info}
           }
intents

{'classifier': <__main__.Classifier at 0x7fa7fb896400>,
 'intents': {0: <__main__.Displayer at 0x7fa7fb8e7e10>,
  1: <__main__.Random_info at 0x7fa7fb896518>}}

##### Chatbot

In [15]:
class ChatBot:
    """ The Chatbot
    """

    def __init__(self, name, intents: dict):
        self.name = name
        self.classifier = intents['classifier']
        self.intents = intents['intents']
        self.active_intent = list(intents.keys())[0]

    def classify(self, sentence: str) -> int:
        """ Choose the best intent """

        self.active_intent = self.classifier.predict(sentence)

    def interact(self, sentence: str) -> str:
        """ Call the interact method of the selected intent 
        and return the sentence generated"""

        self.classify(sentence)
        intent = self.intents[self.active_intent]
        interaction = intent.interact(sentence)
        return interaction

    def synchronize(self, event: str):
        """ synchronize all the intents """

        for intent in self.intents.values():
            intent.synchronize(event)

    def get_interface_in(self) -> str:
        """ get the traduction of the input from the interface
        with the dashboard"""

        return self.intents[self.active_intent].get_interface_in()

    def get_interface_out(self) -> str:
        """ get the traduction of the output for the interface
        with the dashboard"""

        return self.intents[self.active_intent].get_interface_out()


Hubert = ChatBot(name='Chatbot Hubert', intents=intents)
Hubert.classify('aertetreb')
Hubert.active_intent

0

In [16]:
sent = 'Show me the graph of passengrs contentments for Matra  \
    and MATRA and AIRBUS in CONGO from 10/1993 to the end of the year 1946'

output_sentence = Hubert.interact(sent)
input_interfac = Hubert.get_interface_in()
output_interfac = Hubert.get_interface_out()

print("IN :\t", input_interfac, '\n')
print(output_sentence, '\n')
print("OUT :\t", output_interfac, '\n')

Converting to features started.



[{'Show': 'O'}, {'me': 'O'}, {'the': 'O'}, {'graph': 'STAT_B'}, {'of': 'O'}, {'passengers': 'STUDIED_B'}, {'contentment': 'STUDIED_E'}, {'for': 'O'}, {'Matra': 'MANU_B'}, {'and': 'O'}, {'MATRA': 'MANU_B'}, {'and': 'O'}, {'AIRBUS': 'MANU_B'}, {'in': 'O'}, {'CONGO': 'COUN_B'}, {'from': 'O'}, {'10/1993': 'DATE1_B'}, {'to': 'O'}, {'the': 'O'}, {'end': 'O'}, {'of': 'O'}, {'the': 'O'}, {'year': 'O'}, {'1946': 'DATE2_B'}] 

IN :	 {"filters": {"aircraft": [], "category": [], "company": [], "country": ["congo"], "date": ["01101993", "31121946"], "manufacturer": ["airbus", "matra"]}, "tab": "general"} 

We suggest you the seats' study from 01-09-2017 to 30-09-2017 for the manufacturer airbus and for the aircrafta320 in france. If you agree, click on the following link ;) 

OUT :	 {"filters": {"aircraft": ["a320"], "category": [], "company": [], "country": ["france"], "date": ["01092017", "30092017"], "manufacturer": ["airbus"]}, "tab": "seat"} 

